# check piece by piece

In [ ]:
%load_ext Cython

In [ ]:
%%cython 

import time
import numpy as np
cimport numpy as np
import cython
from libc.string cimport memset
from scipy.linalg.cython_blas cimport dsymm, dcopy, dgemm, dgemv, daxpy, dsyrk, \
    dtrmv, dger, dnrm2, ddot, dscal
from scipy.linalg.cython_lapack cimport dpotrf, dpotrs, dpotri, dtrtrs


cdef void comp_summary(int pa, int pan, int nn, int n, int one_int, double one, 
                       double[:,:] Ca, 
                       double[:,:] CC, 
                       double[:] ya,  
                       double Wmij, 
                       double[:,:] SC, double[:] Sy ):

    #SC += CCs[j] * Wm[i,j]            
    daxpy(&nn, &Wmij, &CC[0,0], &one_int, &SC[0,0], &one_int)
    #Sy += yp[b].dot(C[b,:]) * Wm[i,j]            
    dgemv('T', &pa, &n, &Wmij, &Ca[0,0], &pa, &ya[0], &one_int, &one, &Sy[0], &one_int)
    

cdef test_(list grads, int m_, list Cs, double[:,:] Xm, double[:] R, 
           double[:,:,:] CCs, int n, int nn, long[:] pas, long[:] is_, long[:] js_, long[:] inj, 
           list yps, list yfs, double[:,:] Wm):

    cdef double one = 1., two = 2., zero = 0., neg1 = -1.
    cdef int one_int = 1
    cdef int pa, pb, pan, pbn
    
    cdef int i, j, i_, j_, di = len(is_), dj =  len(js_)
    
    cdef double[:,:] tmp_nn = np.zeros( (n,n), order='F')
    cdef double[:,:] tmp_nn2 = np.zeros((n,n), order='F')
    cdef double[:] tmp_n = np.zeros(n)
    
    cdef double[:] Sy, yb, ya, yb_, ya_
    cdef double[:,:] grad_a, grad_b, grad_a_, grad_b_
    cdef double[:,:] Cb, Ca, Cb_, Ca_, CC, SC
    
    for i_ from 0 <= i_ < di:
        
        i = is_[i_]
        
        pa = pas[i]
        pan = pa * n
        
        ya = yfs[i]
        Ca = Cs[i]
        grad_a = grads[i]
        
        SC = np.zeros((n,n))
        Sy = np.zeros(n)        
        for j_ from 0 <= j_ < dj:

            j = js_[j_]        
            pb = pas[j]
            pbn = pb * n
            comp_summary(pb, pbn, nn, n, one_int, one, Cs[j], CCs[:,:,j], yps[j], Wm[i,j], SC, Sy)            
                               
        # tmp_nn = Xm.dot(SC)
        dgemm('T', 'N', &n, &n, &n, &one, &Xm[0,0], &n, &SC[0,0], &n, &zero, &tmp_nn[0,0], &n) 
        # tmp_nn = tmp_nn.dot(Xm.T)
        dgemm('N', 'N', &n, &n, &n, &one, &tmp_nn[0,0], &n, &Xm[0,0], &n, &zero, &tmp_nn2[0,0], &n) 
        # grad[a,:] = grad[a,:] + C[a,:].dot(tmp_nn)
        dgemm('N', 'N', &pa, &n, &n, &one, &Ca[0,0], &pa, &tmp_nn2[0,0], &n, &one, &grad_a[0,0], &pa)        

        # tmp_n = Sy.dot(Xm.T)
        dgemv('T', &n, &n, &one, &Xm[0,0], &n, &Sy[0], &one_int, &zero, &tmp_n[0], &one_int)                       
        # grad[a,:] = grad[a,:] - yf[a].dot(tmp_n.T)
        dger(&pa, &n, &neg1, &ya[0], &one_int, &tmp_n[0], &one_int, &grad_a[0,0], &pa)
        
    for j_ from 0 <= j_ < dj:
        
        j = js_[j_]                
        pb = pas[j]
        pbn = pb * n
                
        yb = yps[j]
        Cb = Cs[j] 
        grad_b = grads[j]
        
        SC = np.zeros((n,n))
        Sy = np.zeros(n)                    
        for i_ from 0 <= i_ < di:

            i = is_[i_]            
            pa = pas[i]
            pan = pa * n
            comp_summary(pa, pan, nn, n, one_int, one, Cs[i], CCs[:,:,i], yfs[i], Wm[i,j], SC, Sy)
                    
        # tmp_nn = Xm.T.dot(SC)
        dgemm('N', 'N', &n, &n, &n, &one, &Xm[0,0], &n, &SC[0,0], &n, &zero, &tmp_nn[0,0], &n) 
        # tmp_nn = tmp_nn.dot(Xm)
        dgemm('N', 'T', &n, &n, &n, &one, &tmp_nn[0,0], &n, &Xm[0,0], &n, &zero, &tmp_nn2[0,0], &n) 
        # grad[b,:] = grad[b,:] + C[b,:].dot(tmp_nn)
        dgemm('N', 'N', &pb, &n, &n, &one, &Cb[0,0], &pb, &tmp_nn2[0,0], &n, &one, &grad_b[0,0], &pb)        

        # tmp_n = Sy.dot(Xm)
        dgemv('N', &n, &n, &one, &Xm[0,0], &n, &Sy[0], &one_int, &zero, &tmp_n[0], &one_int)                       
        # grad[b,:] = grad[b,:] - yp[b].dot(tmp_n.T)
        dger(&pb, &n, &neg1, &yb[0], &one_int, &tmp_n[0], &one_int, &grad_b[0,0], &pb)

    return np.array(grad_a), np.array(grad_b)
        
    """
    cdef int pab, pabn
    cdef double[:] Ranb_, Ranb
    cdef double[:,:] Canb_, Canb, grad_anb_, grad_anb, tmp_anb
    if m_ == 0:
        for i in inj:
            anb = idx_grp[i]
            pab = len(anb)
            pabn = pab * n
            
            Ranb_ = R[anb]
            dcopy(&pab, &Ranb_[0], &one_int, &Ranb[0], &one_int)
            Canb_ = C[anb]
            dcopy(&pabn, &Canb_[0,0], &one_int, &Canb[0,0], &one_int)
            grad_anb_ = grad[anb]
            dcopy(&pabn, &grad_anb_[0,0], &one_int, &grad_anb[0,0], &one_int)
            
            #Ranb = R[anb] * Wm[i,i]
            daxpy(&pab, &Wm[i,j], &Ranb[0], &one_int, &tmp_anb[0], &one_int)
             
            
            # grad[anb,:] += (R[anb]*Wm[i,i]).reshape(-1,1) * (C[anb,:].dot(Xm + Xm.T) 
            # The Hadamard product involving R[anb] is problematic until we correctly 
            
            # ASSUMING Xm = Xm.T !
            #dgemm('N', 'N', &pab, &n, &n, &two, &Cb[0,0], &pab, &Xm[0,0], &n, &one, &grad_anb[0,0], &pb)
    """ 

def test(ts, ms, C, double[:,:] Xm, double[:] R, list idx_grp, get_idx_grp, y, double[:,:] Wm):

    is_, js_, inj = np.array((0,), dtype=int), np.array((1,), dtype=int), np.array([], dtype=int)
    m_ = ms[0]

    cdef int nig = len(idx_grp)
    cdef int n = 20 #Cs[0].shape[1]
    cdef int nn = n * n

    CCs = np.empty((n,n,len(idx_grp)), order='F')
    cdef long[:] pas = np.empty(len(idx_grp), dtype = int)
    cdef int i, di = len(idx_grp)
    Cs = []    
    for i from 0 <= i < di:       # for i in all_is_:
        a = idx_grp[i]
        CCs[:,:,i] = C[a,:].T.dot(C[a,:])
        Cs.append(C[a,:].T.copy() if i in range(len(idx_grp)) else None)
        pas[i] = len(a)    
    
    #CCs = np.empty((n,n,nig), order='F')
    #cdef long[:] pas = np.empty(nig, dtype = int)
    #Cs = []
    #cdef int i
    #for i from 0 <= i < nig:       # for i in all_is_:
    #    CCs[:,:,i] = Cs[i].T.dot(Cs[i])
    #    Cs.append(Cs[i].T.copy() if i in range(nig) else None)
    #    pas[i] = len(idx_grp[i])
        
    grads = [np.zeros((len(a),n), order='F') for a in idx_grp]
    for t in ts:
        out = test_(grads, m_, Cs, Xm, R, CCs, n, nn, pas, is_, js_, inj, 
                    [y[t,a] for a in idx_grp], [y[t+m_,a] for a in idx_grp] , Wm)        
    return out


In [ ]:
import numpy as np
import time
from ssidid.SSID_Hankel_loss import g_C_l2_vector_pair_sso

p,n,T = 1000, 20, 10000

#np.random.seed(0)
C, R = np.random.normal(size=(p,n)), np.random.normal(size=(p,))
Xm = np.random.normal(size=(n,n))

x = np.random.normal(size=(T,n))
y = x.dot(C.T)

#print('y[0]', y[0])

lag_range = np.arange(n)
kl_ = np.max(lag_range)+1
ts, ms = range(T-kl_), (1,)
a,b = np.sort(np.random.choice(p, p//2, replace=False)), np.sort(np.random.choice(p, p//2, replace=False))
#a,b = np.arange(p), np.arange(p)

idx_grp = [a,b]
is_, js_, inj = np.array((0,)), np.array((1,)), []

Wm = 1/T  * np.ones((len(idx_grp), len(idx_grp)))

get_idx_grp = None

t_ = time.time()
Cs = [C[a,:].copy() for a in idx_grp]
out_ = test(ts, ms, C, Xm, R, idx_grp, get_idx_grp, y, Wm)
out = np.zeros((p,n))
for i in range(len(idx_grp)):
    a = idx_grp[i]
    out[a,:] += out_[i] 
print(time.time() - t_)
print('out' , out)

grad = np.zeros((p,n))                                                                   
p,n = C.shape
m_ = ms[0]
t_ = time.time()
for t in ts:
    CCs = [C[a,:].T.dot(C[a,:]) for a in idx_grp]
    g_C_l2_vector_pair_sso(grad, m_, C, Xm, R, CCs, idx_grp, is_, js_, inj, y[t], y[t+m_], Wm)
print(time.time() - t_)
des = grad
print('des', des)